# CORRECCIÓN A IMÁGENES DE CIENCIA

Para trabajar con las imágenes de los cúmulos, primero debemos aplicar las correcciones por dark, flats y la la resta del cielo. 

In [1]:
import sys
import os
import astropy.io.fits as pyfits
from astropy.table import Table
import numpy
import pylab
import itertools
from scipy import stats
from scipy import ndimage
import operator
import time
import datetime
import pickle
import IPython
from astropy.io import fits
from astropy.io import fits
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## REDUCCIÓN DE DARKs


Hacemos una lista con todas las imágenes de darks 

In [ ]:
# ESCRIBIMOS LA DIRECCIÓN DE LAS IMÁGENES DE LA OBSERVACIÓN 
Images_dark=glob("/Volumes/Cami/TESIS/datos_obser_5_marzo/dark/*.fits")
dark = []
archivo_fits_dark=[]

for archivo in tqdm(Images_dark):
    archivo_fits_dark = fits.open(archivo,"readonly",memmap=False) #leer un archivo FITS
    if (archivo_fits_dark['PRIMARY'].header['OBSTYPE'] =='dark'):
        dark.append(archivo_fits_dark)
#nombre['PRIMARY'].header['FILE']
    #print(archivo_fits['PRIMARY'].header['OBSTYPE'])

 Al intentar calcular la mediana, el kernel no lo soporto y se calculará en pequeños segmentos.

In [ ]:
# ESTE TAMAÑO VA CAMBIANDO 
size=250
master_dark = np.zeros((2048,2048))

for i in tqdm(range(0, 2050, size)):
    for j in range(0,2050,size):
        bloque = []       
        #if j%10== 0: print(i,j)
        for archivo in darkss:
            img = fits.getdata(archivo)
            if img.shape !=(2048,2048):
                raise ValueError(f'wrong img shape {img.shape}')
            bloque.append(img[i:(i+size),j:(j+size)])
        #print(np.array(bloque).shape)
        bloque= np.median(bloque, axis=0)    #np.nanmedia
        #print(bloque.shape,np.mean(bloque))
        master_dark[i:(i+size),j:(j+size)]=bloque
        #break

In [ ]:
# CORROBORAR EL master_dark EN UN PLOT.
plt.imshow(master_dark,vmin=-100,vmax=100)
plt.colorbar()

Le cambiamos el nombre de la imagen a masterdark 

In [ ]:
HR=archivo_fits_dark['PRIMARY'].header
HR['OBJECT']='master_dark'

Guardamos el masterdark en un formato FITS

In [ ]:
Hdu = fits.PrimaryHDU(master_dark,HR)
Hdu.writeto('master_dark.fits')

## CORRECCIÓN POR DARK A LAS IMÁGENES DE CIENCIA 

In [ ]:
# ESCRIBIMOS LA DIRECCIÓN DE MASTER_DARK
Master_dark = fits.open("/Volumes/Cami/TESIS/datos_obser_5_marzo/dark/master_dark.fits")
Master_dark.info()
masterdark = Master_dark[0].data

# ESCRIBIMOS LA DIRECCIÓN DE LAS IMÁGENES DE CIENCIAS
direc_ciencia = str('/Volumes/Cami/TESIS/datos_obser_5_marzo/Imgs_ciencia')
os.chdir(direc_ciencia)
os.getcwd()

PRrimero debemos obtener el scaleFactor y para eso debemos obtener los tiempos de exposición (ExpTime) de cada imagen de ciencia

In [ ]:
lista_ciencias = glob("/Volumes/cami/Observacion13enero/Observacion_ciencia/*.fits")
ciencia = []
archivo_fits_ciencia=[]

for archivo in tqdm(lista_ciencias):
    archivo_fits_ciencia = fits.open(archivo,"readonly",memmap=False) #leer un archivo FITS
    ciencia.append(archivo_fits_ciencia)

#TIEMPO DE EXPOSICIÓN DE LAS IMÁGENES CIENCIAS
exptime=[]
for i in range(len(ciencia)):
    exptime.append(ciencia[i]['PRIMARY'].header['EXPTIME'])
#exptime

### scaleFactor

La definición de scaleFactor es:
$$ scaleFactor = \frac{expTime}{darkExpTime}*sarkScaling[nfs]['slope'] + darkScaling[nfs]['intercept']$$

donde $darkScaling[nfs]['slope'] = 1$ y $darkScaling[nfs]['intercept'] = 0 $ y el darkExpTime se busca en la imagen de masterdark(es un valor constante)

In [ ]:
# darkExpTime ES EL TIEMPO DE EXPOSICIÓN DEL MASTERDARK
darkExpTime = 8.733

# CALCULAMOS EL scaleFactor
for i in range(len(exptime)):
    exptime[i] /= 8.733
    
scaleFactor=np.array(exptime)  

## APLICAMOS LA CORRECCIÓN 

In [ ]:
img_ciencia_correc = np.zeros((2048,2048))

for i in tqdm(range(len(lista_ciencias))):
    fits_ciencia = fits.open(lista_ciencias[i])
    img_ciencia = fits_ciencia['PRIMARY'].data
    
    img_ciencia_correc = img_ciencia - (masterdark*scaleFactor[i])
    
    #Guardando las imágenes nuevas
    HR = fits_ciencia['PRIMARY'].header
    Hdu = fits.PrimaryHDU(img_ciencia_correc,HR)
    Hdu.writeto(lista_ciencias[i].replace('.fits','_dark.fits'))

Una vez que se corrigieron los flats por los dark, podemos hacer la reducción para luego aplicarlos a las imágenes de ciencia. 

## REDUCCIÓN DE FLATS

In [2]:
# DIRECTORIO DE LAS IMÁGENES DE CIENCIAS CORREGIDAS POR LOS DARKs
Direc_ciencia=glob("/Volumes/Cami/Observacion13enero/Observacion_ciencia/*_dark.fits")

Hacemos una lista donde solo se encuentren los flats y le aplicamos el promedio a todas esas imágenes. 

In [ ]:
flats = []
archivo_fits_flats = []

for archivo in tqdm(Direc_ciencia):
    archivo_fits_flats = fits.open(archivo,"readonly",memmap=False) #leer un archivo FITS
    if (archivo_fits_flats['PRIMARY'].header['OBSTYPE'] =='dflat'):
        img = fits.getdata(archivo)
        flats.append(img)
master_flats = np.nanmean(flats,axis=0)
    #archivo_fits.close()

In [ ]:
## PARA CORROBORAR EL master_flats EN UN PLOT.
plt.imshow(master_flats,vmin=14000,vmax=18000)
plt.colorbar()

Le cambiamos el nombre de la imagen a masterflats

In [ ]:
hr=archivo_fits_flats['PRIMARY'].header
hr['OBJECT']='master_flats'

Guardamos el masterflats en un formato FITS

In [ ]:
hdu = fits.PrimaryHDU(master_flats,hr)
hdu.writeto('master_flats.fits')

## CORRECCIÓN POR FLATS A LAS IMÁGENES DE CIENCIA

In [ ]:
# ESCRIBIMOS LA DIRECCIÓN DE MASTER_FLATS
Master_flats = fits.open("/Volumes/Cami/TESIS/datos_obser_5_marzo/dark/master_flats.fits")
Master_flats.info()
masterflats = Master_flats[0].data

# ESCRIBIMOS LA DIRECCIÓN DE LAS IMÁGENES DE CIENCIAS
Direc_ciencia=glob("/Volumes/Cami/Observacion13enero/Observacion_ciencia/*_dark.fits")
os.chdir(Direc_ciencia)
os.getcwd()

En la primera prueba dan valores 0 en el masterflat y para esto se reemplaza  los ceros por el promedio de los píxeles vecinos.

In [ ]:
# LOS n_x y n_y SON EL TAMAÑOS DE LA IMAGEN 
n_x = n_y = 2048         
for i in range(n_x):
    for j in range(n_y):
        if masterflat[i,j] == 0:
            masterflat[i,j] = masterflat[i-1:i+2,j-1:j+2].mean()
#(masterflat==0).sum()

APLICAMOS LA CORRECCIÓN

In [ ]:
img_ciencia_corr_flats = np.zeros((2048,2048))
mean_MF = masterflat.mean()
img_astro=[]

for i in tqdm(range(len(Direc_ciencia))):
    fits_ciencia_astro = fits.open(Direc_ciencia[i])
    img_ciencia = fits_ciencia_astro['PRIMARY'].data 
    
    if (fits_ciencia_astro['PRIMARY'].header['OBSTYPE']=='astro'):
        img=fits.getdata(Direc_ciencia[i])
        
        img_ciencia_corr_flats =(img/masterflat)*mean_MF
 
        #Guardando las imágenes nuevas
        IMG = fits_ciencia_astro['PRIMARY'].header
        Hdu = fits.PrimaryHDU(img_ciencia_corr_flats,IMG)
        Hdu.writeto(Direc_ciencia[i].replace('.fits','_flats.fits'))

Probemos en un plot el img_ciencia_corr_flats

In [ ]:
plt.imshow(img_ciencia_corr_flat, vmin=5000,vmax=8000)
plt.colorbar()

## RESTA DEL CIELO

In [ ]:
# DEFINO EL LUGAR DONDE SE ENCUENTRAN LAS IMÁGENES DE CIENCIA
direc_img_ciencia = glob("/Volumes/Cami/TESIS/datos_obser_5_marzo/Imgs_ciencia/*_dark_flats.fits")

In [ ]:
# HACEMOS LA SEPARACIÓN DE LAS IMÁGENES POR EL FILTRO Ks (TAMBIÉN PUEDE SER CON EL FILTRO J)
filter_Ks = []
for archivo in tqdm(direc_img_ciencia):
    filter_1 = fits.getheader(archivo)  #leer un archivo FITS
    #print(HR['EXPTIME'])
    #print(type(HR['EXPTIME']))
    if (filter_1['FILTER'] == 'Ks'):
        filter_Ks.append(archivo)

In [ ]:
# HACEMOS LA SEPARACIÓN POR EL MISMO Loop 
loops=[]
for archivo in tqdm(filter_Ks):
    archivo_fits = fits.open(archivo,"readonly",memmap=False) #leer un archivo FITS
    if (archivo_fits['PRIMARY'].header['LOOP'] == 1):
        img = fits.getdata(archivo)
        loops.append(img)

In [ ]:
# SACAMOS LA MEDIANA DE LAS IMÁGENES DE CIENCIA 
subtractSky = np.median(loops,axis=0)

In [ ]:
img_ciencia_correc_sky = np.zeros((2048,2048))

for i in tqdm(range(len(direc_img_ciencia))):
    img_sky = fits.open(direc_img_ciencia[i], "readonly",memmap = False)
    Img_Sky = img_sky['PRIMARY'].data
    
    ## imágenes corregidas
    
    img_ciencia_correc_sky = Img_Sky - subtractSky[i]
        
    #Guardando las imágenes nuevas
   
    HR = img_sky['PRIMARY'].header
    Hdu = fits.PrimaryHDU(img_ciencia_correc_sky,HR)
    Hdu.writeto(direc_img_ciencia[i].replace('.fits','_sky.fits'))

In [ ]:
# PARA CORROBORAR, HACEMOS UN PLOTS
plt.imshow(img_ciencia_correc_sky, vmin=2,vmax=5)
plt.colorbar()